In [2]:
import urllib

params = 'DRIVER={ODBC Driver 17 for SQL Server};' \
         'SERVER=localhost;' \
         'PORT=1433;' \
         'DATABASE=DbPmrs;' \
         'UID=sa;' \
         'PWD=aSj#14776$10036%mM&;'
            
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine

db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

import pandas as pd

sql = '''
SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_CATALOG='DbPmrs'
'''

pd.read_sql_query(sql, db)

,TABLE_NAME
0,tblw_DateConversion
1,tblw_ReportConfirm
2,tblw_ReportDox
3,tblw_Machinary
4,tblw_ZoneImage
...,...
58,tblw_TimeProgressState
59,tbl_JContractReportDate
60,tbl_UserLogin
61,tbl_JUserRole


In [ ]:
import datetime
import math

dt = datetime.datetime(2023,7,9)
print('date=', dt.timestamp())

y = (math.trunc(dt.timestamp()) + 467066.53004084 - 0.641087919916919581508) / 365.24219878
print('y=', y)
y1 = y - math.trunc(y)
print('y1=', y1)
print('cond1=', (y1 <= 0.084875187 * 6))
print('cond2=', y1 <= (0.084875187 * 6 + 0.082137278 * 5))

m = 1 + math.trunc((y1 / 0.084875187)) if (y1 <= 0.084875187 * 6) else \
    (7 + math.trunc(((y1 - 0.084875187 * 6) / 0.082137278)) \
    if y1 <= (0.084875187 * 6 + 0.082137278 * 5) else 12)
print('m=',m)
mm = '0' + str(math.trunc(datetime.datetime.fromtimestamp(m))) if m < 10 else \
    str(math.trunc(datetime.datetime.fromtimestamp(m)))

d = math.trunc(datetime.datetime.fromtimestamp(y1 * 365.24219878 - (m - 1) * 31)) + 1 if (m <= 6) else \
    math.trunc(datetime.datetime.fromtimestamp(y1 * 365.24219878 - (m - 1) * 30 - 6)) + 1 if (m < 12) else \
    math.trunc(datetime.datetime.fromtimestamp(y1 * 365.24219878 - 336)) + 1

dd = '0' + str(math.trunc(datetime.datetime.fromtimestamp(d))) if d < 10 else \
    str(math.trunc(datetime.datetime.fromtimestamp(d)))

result = str(math.trunc(datetime.datetime.fromtimestamp(y))) + '/' + mm + '/' + dd

print(result)

In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from contracts.models import *
import jdatetime
from datetime import datetime

date = datetime(2021, 2, 19)

jDate = jdatetime.date.fromgregorian(day=date.day,month=date.month,year=date.year)

print('%s-%s-%s' % (str(jDate.year), str(jDate.month), str(jDate.day)))
# x = TblContract.objects.get(pk=123476)
# x.startOperationShamsiDate()

# print('incrementalAmount_sum=', incrementalAmount_sum)

1399-12-1


In [3]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django.contrib.auth import get_user_model
from accounts.models import TblUser

puser = get_user_model()
tusers = TblUser.objects.all()

for tuser in tusers:
    user = puser(
                password = 'pbkdf2_sha256$260000$7k87SmvNjAwEqa1l0MHT46$CKNzNbFWmRykPWl7BrzYzWcRwPSvIxAJnxJwxjrxX1o=',
                username = tuser.user,
                first_name = '',
                last_name = '',
                email = '',
                is_active = tuser.active,
                priority = tuser.priority
                )
    user.save()
    user.set_password('654321')

pu = get_user_model().objects.all()
print(len(pu))

ImportError: cannot import name 'TblUser' from 'accounts.models' (D:\projects\cost_control\cost-control-backend\main\accounts\models.py)

In [2]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from accounts.serializers import LoginSerializer

try:
    data = {
        "username": "h.masajedian",
        "password": "ad_min123"
    }    
    serializer = LoginSerializer(data=data)
    print(serializer.data)

    serializer.is_valid(raise_exception=True)
    user = serializer.validated_data
    print(user)
    
    _, token = AuthToken.objects.create(user)
    print(token)
except Exception as e:
    print(str(e))




When a serializer is passed a `data` keyword argument you must call `.is_valid()` before attempting to access the serialized `.data` representation.
You should either call `.is_valid()` first, or access `.initial_data` instead.


In [31]:
from django.db.models.expressions import Window
from django.db.models.functions import RowNumber
from django.db.models import F

result = (
    TblwProblem.objects.order_by("problemid", "contractid", "dateid")
    .annotate(
        row_number=Window(
            expression=RowNumber(), partition_by=[F("contractid")], order_by=[F("problemid")]
        )
    )
    .values("problemid", "contractid", "dateid", "problem", "row_number")
)
print(result)

<QuerySet [{'problemid': 1019, 'contractid': 1234, 'dateid': 1051, 'problem': 'نیاز به تزریق مناسب مالی جهت تکمیل قراردادهای ساخت تجهیزات.\t\t\t\t\t\t\t\t\t\t\t', 'row_number': 1}, {'problemid': 1020, 'contractid': 1234, 'dateid': 1051, 'problem': 'تصمیم گیری در خصوص مبالغ کارهای اضافه و ابلاغ آن از سوی کارفرما و تهیه الحاقیه قراردادی مربوطه با کارفرما', 'row_number': 2}, {'problemid': 1021, 'contractid': 1234, 'dateid': 1051, 'problem': 'تاثیر افزایش نرخ ارز در فعال نمودن قراردادهای جدید', 'row_number': 3}, {'problemid': 1023, 'contractid': 123465, 'dateid': 1051, 'problem': 'عدم ابلاغ  نقشه هاي فيبر نوري', 'row_number': 1}, {'problemid': 1027, 'contractid': 123464, 'dateid': 1051, 'problem': 'عدم تامین به موقع لوله های باقی مانده ( طبق برنامه زمانبندی پروژه متریال Piping از قبیل Flange - Bolt - Gasket - Mating Flange - Special Part1-2 می بایست در سایت باشد. با توجه به مسیر بحرانی پروژه راه اندازی با تاخیر انجام خواهد شد)', 'row_number': 1}, {'problemid': 1028, 'contractid': 123464, '

In [32]:
from projects.models import *
from projects.serializers import *

contractId = 123472
dateId = 1101
problems = TblwProblem.objects.filter(contractid__exact=contractId, dateid__exact=dateId)
# problems = TblwProblem.row_number_objects.filter(contractid__exact=contractId, dateid__exact=dateId)
serializer = ProblemSerializer(problems, many=True)
 

serializer.data
# problems

[OrderedDict([('problemid', 2894), ('contractid', 123472), ('dateid', 1101), ('problem', 'عدم اجازه ترخیص و نصب کنترل ولوهای ارسال شده به کارگاه')]), OrderedDict([('problemid', 2895), ('contractid', 123472), ('dateid', 1101), ('problem', 'عدم تأیید و ابلاغ ساختار شکست بخش  راه اندازی پروژه')]), OrderedDict([('problemid', 2896), ('contractid', 123472), ('dateid', 1101), ('problem', 'عدم تأیید ساختار شکست اقلام مربوط به لوازم یدکی،مواد مصرفی و ابزار اختصاصی برای دوران راه اندازی و دوره 2 ساله بهره برداری')]), OrderedDict([('problemid', 2897), ('contractid', 123472), ('dateid', 1101), ('problem', 'عدم تأیید و پرداخت مبالغ بخش جبران هزینه های ناشی از کرونا')]), OrderedDict([('problemid', 2904), ('contractid', 123472), ('dateid', 1101), ('problem', 'problem1')])]

In [5]:
from django.db.models.functions import Substr
from contracts.models import TblContract
from contracts.services import GregorianToShamsi
from projects.models import *

# dateId = 0
max_date_id = TblwReportdate.objects.aggregate(Max('dateid'))['dateid__max']
date = TblwReportdate.objects.get(pk=max_date_id)
y1 = int(date.year)
m1 = int(date.month)

now = GregorianToShamsi(datetime.now())
y2 = int(now[0:4])
m2 = int(now[5:now.find('-', 5)])

dates = TblwReportdate.objects.filter(year=str(y2), month=str(m2))
dateId = dates[0] if dates is not None and len(dates) > 0 else 0
print('dateId: ', dateId)

loop = 0
if((y2 - y1) > 1 or ((y2 - y1) == 0 and (m2 - m1) > 0) or ((y2 -y1) == 1 and (12 - m1 + m2 > 0))):
    loop = 1
print('loop: ', loop)

while(loop == 1):
    if(y2 == y1):
        m1 = m1 + 1
        date = TblwReportdate.objects.create(year=str(y1), month=str(m1))
        dateId = date.dateid

        flag = 0
        contracts = TblContract.objects.exclude(contract__exact='test').filter(iscompleted__exact=False)
        for contract in contracts:
            flag = TblJcontractreportdate.objects.filter(contractid__exact=contract.contractid, 
                                                  dateid__year__exact=y2,
                                                  dateid__month__exact=m2).count()
            if flag == 0:
                TblJcontractreportdate.objects.create(contractid=contract, dateid=date)

        if(m2 == m1):
            loop = 0

    elif(y2 - y1 > 0):
        m1 = m1 + 1
        if(m1 > 12):
            m1 = 1
            y1 = y1 + 1

        date = TblwReportdate.objects.create(year=str(y1), month=str(m1))
        dateId = date.dateid

        flag = 0
        contracts = TblContract.objects.exclude(contract__exact='test').filter(iscompleted__exact=False)
        for contract in contracts:
            flag = TblJcontractreportdate.objects.filter(contractid__exact=contract.contractid, 
                                                  dateid__year__exact=y2,
                                                  dateid__month__exact=m2).count()
            if flag == 0:
                TblJcontractreportdate.objects.create(contractid=contract, dateid=date)

        if(y2 == y1 and m2 == m1):
            loop = 0


dateId:  0
loop:  1


In [3]:
import pandas as pd
from django_pivot.pivot import pivot
from django.db.models import F
from contracts.models import TblCorporation

contractCorporations = TblCorporation.objects.filter(contractid__exact=123467).values(
    'companyid__company', 'e_percent', 'p_percent', 'c_percent').annotate(company = F('companyid__company')).values(
    'company', 'e_percent', 'p_percent', 'c_percent')
for row in contractCorporations:
    print(row)
print('==============================================')
pivot_table1 = pivot(contractCorporations, 'e_percent', 'companyid', 'e_percent')
pivot_table1 = pivot_table1.exclude(
#     e_percent__exact=None).exclude(
    e_percent__exact=0.0)

pivot_table2 = pivot(contractCorporations, 'p_percent', 'companyid', 'p_percent')
pivot_table2 = pivot_table2.exclude(
#     p_percent__exact=None).exclude(
    p_percent__exact=0.0)

pivot_table3 = pivot(contractCorporations, 'c_percent', 'companyid', 'c_percent')
pivot_table3 = pivot_table3.exclude(
#     c_percent__exact=None).exclude(
    c_percent__exact=0.0)

result = pivot_table1.union(pivot_table2).union(pivot_table3)

# for row in coms:
#     print(row.companyid, '   ,', row.company)

for row in pivot_table1:
    print(row)
for row in pivot_table2:
    print(row)
for row in pivot_table3:
    print(row)

# df = pd.DataFrame(list(contractCorporations.values('companyid_id', 'e_percent', 'p_percent', 'c_percent')))
# print(df)
# print('==============================================')
# # pivot_table = df.pivot(index = ['e_percent', 'p_percent', 'c_percent'], columns=['companyid_id'])
# pivot_table = pd.pivot_table(df, index = ['e_percent', 'p_percent', 'c_percent'])
# print(pivot_table)


{'e_percent': 0.0, 'p_percent': 0.0, 'c_percent': 100.0, 'company': 'آسفالت طوس'}
{'e_percent': 100.0, 'p_percent': 50.0, 'c_percent': 0.0, 'company': 'فکور صنعت تهران'}
{'e_percent': 0.0, 'p_percent': 50.0, 'c_percent': 0.0, 'company': 'ماشین سازی ویژه'}
{'e_percent': 100.0, '7': None, '1021': 100.0, '1024': None}
{'p_percent': 50.0, '7': None, '1021': 50.0, '1024': 50.0}
{'c_percent': 100.0, '7': 100.0, '1021': None, '1024': None}


In [35]:
from contracts.models import TblCorporation

result = TblCorporation.objects.filter(contractid__exact=123467)
#print(result)
for row in result:
    print(row)

آسفالت طوس: e=0.0, p=0.0, c=100.0
فکور صنعت تهران: e=100.0, p=50.0, c=0.0
ماشین سازی ویژه: e=0.0, p=50.0, c=0.0


In [8]:

import pandas as pd
# Create a DataFrame
df = pd.DataFrame({'Gender' : ['Female', 'Male', 'Male', 'Male', 'Female'],
                  'Courses': ['Java', 'Spark', 'PySpark', 'C', 'Pandas'],
                   'Fee': [15000, 17000, 27000, 29000, 12000],
                   'Discount': [1100, 800, 1000, 1600, 600]})
print(df)
p_table = pd.pivot_table(df, index = ['Gender', 'Courses', 'Fee'])
print(p_table)

   Gender  Courses    Fee  Discount
0  Female     Java  15000      1100
1    Male    Spark  17000       800
2    Male  PySpark  27000      1000
3    Male        C  29000      1600
4  Female   Pandas  12000       600
                      Discount
Gender Courses Fee            
Female Java    15000      1100
       Pandas  12000       600
Male   C       29000      1600
       PySpark 27000      1000
       Spark   17000       800


In [6]:
from contracts.models import TblCorporation
#[0]['e_percent']
e = TblCorporation.objects.filter(
    contractid__exact=123467).values('companyid__company')
print (e)

<QuerySet [{'companyid__company': 'آسفالت طوس'}, {'companyid__company': 'فکور صنعت تهران'}, {'companyid__company': 'ماشین سازی ویژه'}]>


In [10]:
from django.db.models import F
from contracts.models import TblCorporation

e = TblCorporation.objects.filter(contractid__exact=123467).values(
    'companyid__company', 'e_percent', 'p_percent', 'c_percent').annotate(value = F('e_percent'),
                                                                         label = F('companyid__company')).values(
    'value', 'label')
print(e)
for row in e:
    print(row)

<QuerySet [{'value': 0.0, 'label': 'آسفالت طوس'}, {'value': 100.0, 'label': 'فکور صنعت تهران'}, {'value': 0.0, 'label': 'ماشین سازی ویژه'}]>
{'value': 0.0, 'label': 'آسفالت طوس'}
{'value': 100.0, 'label': 'فکور صنعت تهران'}
{'value': 0.0, 'label': 'ماشین سازی ویژه'}


In [6]:
from contracts.models import TblCorporation
from contracts.serializers import ContractCorporationSerializer

Corporations = TblCorporation.objects.filter(contractid__exact=123467)

serializer = ContractCorporationSerializer(Corporations, many=True)

serializer.data


AttributeError: 'QuerySet' object has no attribute 'items'

In [5]:
from django_pivot.pivot import pivot
from contracts.models import TblCorporation

contractCorporations = TblCorporation.objects.filter(contractid__exact=123467)

pivot_table1 = pivot(contractCorporations, 'e_percent', 'companyid', 'e_percent')
pivot_table1 = pivot_table1.exclude(e_percent__exact=0.0)

pivot_table2 = pivot(contractCorporations, 'p_percent', 'companyid', 'p_percent')
pivot_table2 = pivot_table2.exclude(p_percent__exact=0.0)

pivot_table3 = pivot(contractCorporations, 'c_percent', 'companyid', 'c_percent')
pivot_table3 = pivot_table3.exclude(c_percent__exact=0.0)

result = pivot_table1.union(pivot_table2).union(pivot_table3)

for row in result:
    print(row)



{'e_percent': 50.0, '7': None, '1021': 50.0, '1024': 50.0}
{'e_percent': 100.0, '7': None, '1021': 100.0, '1024': None}
{'e_percent': 100.0, '7': 100.0, '1021': None, '1024': None}


In [6]:
from django_pivot.pivot import pivot
from contracts.models import TblCorporation

contractCorporations = TblCorporation.objects.filter(contractid__exact=123467)

pivot_table1 = pivot(contractCorporations, 'e_percent', 'companyid__company', 'e_percent')
pivot_table1 = pivot_table1.exclude(e_percent__exact=0.0)

pivot_table2 = pivot(contractCorporations, 'p_percent', 'companyid__company', 'p_percent')
pivot_table2 = pivot_table2.exclude(p_percent__exact=0.0)

pivot_table3 = pivot(contractCorporations, 'c_percent', 'companyid__company', 'c_percent')
pivot_table3 = pivot_table3.exclude(c_percent__exact=0.0)

result = pivot_table1.union(pivot_table2).union(pivot_table3)

for row in result:
    print(row)

ValueError: Column aliases cannot contain whitespace characters, quotation marks, semicolons, or SQL comments.

In [7]:
from contracts.models import TblContract
from datetime import datetime

contractId = 123477
date = "2020-08-31"

date_format = "%Y-%m-%d"
startOperationDate = datetime.strptime(str(date), date_format)

TblContract.objects.filter(contractid__exact=contractId).update(startoperationdate=startOperationDate)
result = TblContract.objects.get(pk=contractId)
print(result)


احداث واحد اکسیژن طرح یزد یک


In [8]:
from projects.models import TblwFinancialinfo
from projects.serializers import FinancialInfoSerializer

contractId = 123477
dateId = 1116
financialinfos = TblwFinancialinfo.objects.filter(contractid__exact=contractId, dateid__exact=dateId)
print(financialinfos)

serializer = FinancialInfoSerializer(instance=financialinfos[0] if len(financialinfos) > 0 else 0, many=False)

print(serializer.data)


<QuerySet []>


AttributeError: Got AttributeError when attempting to get a value for field `contractid` on serializer `FinancialInfoSerializer`.
The serializer field might be named incorrectly and not match any attribute or key on the `int` instance.
Original exception text was: 'int' object has no attribute 'contractid'.

In [19]:
from accounts.models import TblJuserrole
from contracts.models import TblContract
from contracts.serializers import ContractSerializerEx

userid = 35
all_contracts = TblJuserrole.objects.filter(userid__exact=userid, contractid__exact=None)
print(all_contracts)
print(len(all_contracts))
      
if(len(all_contracts) == 1):
    contracts = TblContract.objects.all().order_by('-startdate')
    serializer = ContractSerializerEx(contracts, many=True)
else:
    contracts = TblContract.objects.filter(Contract_UserRole__userid__exact=userid).order_by(
        '-startdate').distinct()
    serializer = ContractSerializerEx(contracts, many=True)
print(serializer.data)

<QuerySet []>
0
[OrderedDict([('contractid', 123469), ('contract', 'پالایشگاه نفت سنگین قشم')]), OrderedDict([('contractid', 123467), ('contract', 'گندله سنگان')]), OrderedDict([('contractid', 123468), ('contract', 'GOSP')]), OrderedDict([('contractid', 1234), ('contract', 'گندله سازی گهر زمین')]), OrderedDict([('contractid', 123465), ('contract', 'پروژه انتقال آب خلیج فارس')]), OrderedDict([('contractid', 123461), ('contract', 'سکوی سلمان')])]


In [5]:
from projects_files.models import HseReportDox

result = hseReportDox = HseReportDox.objects.all()
print(result)

ImportError: cannot import name 'HseReportDox' from 'projects_files' (D:\projects\cost_control\cost-control-backend\main\projects_files\__init__.py)

In [10]:
from projects.models import ReportDate
from projects_files.models import Zone, ZoneImage
from projects_files.serializers import ZoneSerializers

contractId = 123477
dateId = 1116

last_dateId = ReportDate.objects.filter(dateid__lt=dateId).aggregate(Max('dateid'))['dateid__max']
print(last_dateId)
count = ZoneImage.objects.filter(zoneid__contractid__exact=contractId, dateid__exact=dateId).count()
print(count)
if count == 0:
    zones = Zone.objects.filter(contractid__exact=contractId, Zone_Zoneimage__dateid__exact=last_dateId) 
                                    #  dateid__exact=last_dateId).exclude(Q(img1__exact=None) | 
                                    #                            Q(img2__exact=None) | 
                                    #                            Q(img3__exact=None)).values('zoneid')
                
    print(len(zones))
    date = ReportDate.objects.get(pk=dateId)
    for zone in zones:
        ZoneImage.objects.create(zoneid=zone, dateid=date)

zoneImage = ZoneImage.objects.filter(zoneid__contractid__exact=contractId, dateid__exact=dateId)
serializer = ZoneSerializers(instance=zoneImage, many=True)
print(serializer.data)

1115
0
0
[]


In [3]:
from projects_files.models import InvoiceDox
from projects_files.serializers import ApprovedInvoiceDoxSerializers

contractId = 123476
dateId = 1117
approvedInvoiceDox = InvoiceDox.objects.filter(contractid__exact=contractId, dateid__exact=dateId)
serializer = ApprovedInvoiceDoxSerializers(instance=approvedInvoiceDox, many=True)
print(serializer.data)

[]


In [4]:
from contracts.models import Contract
from projects.models import ReportConfirm
from contracts.serializers import ContractBaseInfoSerializer

contractId = 123477
dateId = 1119

contractBaseInfo = Contract.objects.get(pk=contractId)
serializer = ContractBaseInfoSerializer(instance=contractBaseInfo, many=False)

reportConfirmed = ReportConfirm.objects.filter(contractid__exact=contractId, dateid__exact=dateId, pm_c__gt=0)

print(serializer.data)
print(reportConfirmed)
# if reportConfirmed is not None and len(reportConfirmed) > 0:
#     print(reportConfirmed[0])


DoesNotExist: Contract matching query does not exist.

In [5]:
from django.contrib.auth import get_user_model
from accounts.models import UserRole
from projects_files.models import *
from projects_files.serializers import *

contractId = 123477
dateId = 1119
userId = 99053
reportId = 1

user = get_user_model().objects.get(pk=userId)
date = ReportDate.objects.get(pk=dateId)

if(contractId == -1):
    reportVisits = ReportVisit.objects.filter(userid__exact=userId, dateid__exact=dateId)
    reportVisits.update(imagereport=1)

    isAllProject = UserRole.objects.filter(userid__exact=userId, contractid__exact=None).count() > 0

    if isAllProject:
        contracts = Contract.objects.filter(iscompleted__exact=False).exclude(contractid__in=reportVisits)
        for contract in contracts:
            ReportVisit.objects.create(contractid=contract, dateid=date, userid=user, imagereport=1)
    else:
        userContractRole = UserRole.object.filter(userid__exact=userId)
        contracts = Contract.objects.filter(contractid__in=userContractRole).exclude(contractid__in=reportVisits)
        for contract in contracts:
            ReportVisit.objects.create(contractid=contract, dateid=date, userid=user, imagereport=1)
else: 
    reportVisits = ReportVisit.objects.filter(userid__exact=userId, contractid__exact=contractId, dateid__exact=dateId)
    contracts = Contract.objects.filter(contractid__exact=contractId)
    if len(contracts) == 1:
        if len(reportVisits) == 0:
            if reportId == 1:
                reportVisit = ReportVisit.objects.create(userid=user, contractid=contracts[0], 
                                           dateid=date, financialinfo=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 2:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid=contracts[0], 
                                           dateid__exact=date, hse=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 3:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=date, progressstate=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 4:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, timeprogressstate__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 5:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, invoice__exact=1)

                ReportVisitdate.object.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 6:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, financialinvoice__exact=1)

                ReportVisitdate.object.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 7:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, workvolume__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 8:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, pmsprogress__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 9:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, budget__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)

            elif reportId == 10:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, machinary__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 11:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, personel__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 12:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, problems__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 13:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, criticalactions__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)

            elif reportId == 14:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, zoneimages__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 15:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, projectdox__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 16:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, durationdox__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 17:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, dashboard_r__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)

            elif reportId == 18:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, dashboard_fc__exact=1)

                ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)
            elif reportId == 19:
                reportVisit = ReportVisit.objects.create(userid__exact=user, contractid__exact=contracts[0], 
                                           dateid__exact=dateId, imagereport__exact=1)

                ReportVisitdate.object.create(visitreportid=reportVisit, reportid=reportId)
        elif len(reportVisits) == 1:
            if reportId == 1:
                reportVisits[0].financialinfo = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 2:
                reportVisits[0].hse = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 3:
                reportVisits[0].progressstate = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 4:
                reportVisits[0].timeprogressstate = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 5:
                reportVisits[0].invoice = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 6:
                reportVisits[0].financialinvoice = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 7:
                reportVisits[0].workvolume = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 8:
                reportVisits[0].pmsprogress = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 9:
                reportVisits[0].budget = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)

            elif reportId == 10:
                reportVisits[0].machinary = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 11:
                reportVisits[0].personel = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 12:
                reportVisits[0].problems = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 13:
                reportVisits[0].criticalactions = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 14:
                reportVisits[0].zoneimages = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 15:
                reportVisits[0].projectdox = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 16:
                reportVisits[0].durationdox = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 17:
                reportVisits[0].dashboard_r = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)

            elif reportId == 18:
                reportVisits[0].dashboard_fc = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
            elif reportId == 19:
                reportVisits[0].imagereport = 1
                reportVisits[0].save()

                ReportVisitdate.objects.create(visitreportid=reportVisits[0], reportid=reportId)
                

        

In [4]:
from django.contrib.auth import get_user_model
from accounts.models import UserRole
from projects_files.models import *
from projects_files.serializers import *

contractId = 123477
dateId = 1119
userId = 99053
reportId = 1

user = get_user_model().objects.get(pk=userId)
date = ReportDate.objects.get(pk=dateId)

# if(contractId == -1):
#     reportVisits = ReportVisit.objects.filter(userid__exact=userId, dateid__exact=dateId)
#     reportVisits.update(imagereport=1)

#     isAllProject = UserRole.objects.filter(userid__exact=userId, contractid__exact=None).count() > 0

#     if isAllProject:
#         contracts = Contract.objects.filter(iscompleted__exact=False).exclude(contractid__in=reportVisits)
#         for contract in contracts:
#             ReportVisit.objects.create(contractid=contract, dateid=date, userid=user, imagereport=1)
#     else:
#         userContractRole = UserRole.object.filter(userid__exact=userId)
#         contracts = Contract.objects.filter(contractid__in=userContractRole).exclude(contractid__in=reportVisits)
#         for contract in contracts:
#             ReportVisit.objects.create(contractid=contract, dateid=date, userid=user, imagereport=1)
# else: 

reportVisits = ReportVisit.objects.filter(userid__exact=userId, contractid__exact=contractId, dateid__exact=dateId)
contracts = Contract.objects.filter(contractid__exact=contractId)
if len(contracts) == 1:
    if len(reportVisits) == 0:
        if reportId == 1:
            reportVisit = ReportVisit.objects.create(userid=user, contractid=contracts[0], 
                                       dateid=date, financialinfo=1)

            ReportVisitdate.objects.create(visitreportid=reportVisit, reportid=reportId)

AttributeError: type object 'ReportVisitdate' has no attribute 'object'

In [21]:
from projects_files.models import ZoneImage
from projects_files.serializers import ReportZoneImagesSerializers
from django.db.models import F

zoneImages1 = ZoneImage.objects.filter(zoneid__exact=21).values(
    'zoneid__zone', 'img1', 'description1').annotate(
    zone=F('zoneid__zone'), 
    img=F('img1'), 
    description=F('description1')
).values('zone', 'img', 'description')
zoneImages2 = ZoneImage.objects.filter(zoneid__exact=21).values(
    'zoneid__zone', 'img2', 'description2').annotate(
    zone=F('zoneid__zone'), 
    img=F('img2'), 
    description=F('description2')
).values('zone', 'img', 'description')
zoneImages3 = ZoneImage.objects.filter(zoneid__exact=21).values(
    'zoneid__zone', 'img3', 'description3').annotate(
    zone=F('zoneid__zone'), 
    img=F('img3'), 
    description=F('description3')
).values('zone', 'img', 'description')

zoneImages = zoneImages1.union(zoneImages2).union(zoneImages3)
for item in zoneImages:
    print(item)
    
serializer = ReportZoneImagesSerializers(zoneImages)
    
print(serializer.data)


{'zone': 'zone1', 'img': '', 'description': ''}
{'zone': 'zone1', 'img': 'zone_images/cell-icon1.png', 'description': 'test11'}
{'zone': 'zone1', 'img': 'zone_images/cell-icon1.png', 'description': 'test2'}
{'zone': 'zone1', 'img': 'zone_images/HSE.png', 'description': 'test1'}
{'zone': 'zone1', 'img': 'zone_images/linkedin-icon.png', 'description': 'test1'}
{'zone': 'zone1', 'img': 'zone_images/linkedin-icon.png', 'description': 'test33'}
{'zone': 'zone1', 'img': 'zone_images/location-icon.png', 'description': 'test3'}
{'zone': 'zone1', 'img': 'zone_images/skype-icon.jpg', 'description': 'test22'}


AttributeError: Got AttributeError when attempting to get a value for field `zone` on serializer `ReportZoneImagesSerializers`.
The serializer field might be named incorrectly and not match any attribute or key on the `QuerySet` instance.
Original exception text was: 'QuerySet' object has no attribute 'zone'.

In [4]:
from projects.models import ReportDate
from projects_files.models import Zone, ZoneImage
from projects_files.services import SetReportVisit

userId = 99053
contractId = 123476
dateId = 1119
reportId = 14

SetReportVisit(userId, contractId, dateId, reportId) 

last_dateId = ReportDate.objects.filter(dateid__lt=dateId).aggregate(Max('dateid'))['dateid__max']

count = ZoneImage.objects.filter(zoneid__contractid__exact=contractId, dateid__exact=dateId)

if count == 0:
    zones = Zone.objects.filter(contractid__exact=contractId, Zone_Zoneimage__dateid__exact=last_dateId) 
                                    #  dateid__exact=last_dateId).exclude(Q(img1__exact=None) | 
                                    #                            Q(img2__exact=None) | 
                                    #                            Q(img3__exact=None)).values('zoneid')

    date = ReportDate.objects.get(pk=dateId)
    for zone in zones:
        ZoneImage.objects.create(zoneid=zone, dateid=date)

zoneImage = ZoneImage.objects.filter(zoneid__contractid__exact=contractId, dateid__exact=dateId)
serializer = ZoneImagesSerializers(instance=zoneImage, many=True)


AttributeError: type object 'ZoneImage' has no attribute 'objects'

In [4]:
from datetime import datetime
from projects.models import *
from projects.serializers import *

max_date_id = ReportDate.objects.aggregate(Max('dateid'))['dateid__max']
date = ReportDate.objects.get(pk=max_date_id)
y1 = int(date.year)
m1 = int(date.month)

now = GregorianToShamsi(datetime.now())
y2 = int(now[0:4])
m2 = int(now[5:now.find('-', 5)])

loop = 0
if((y2 - y1) > 1 or ((y2 - y1) == 0 and (m2 - m1) > 0) or ((y2 -y1) == 1 and (12 - m1 + m2 > 0))):
    loop = 1

while(loop == 1):
    if(y2 == y1):
        m1 = m1 + 1
        date = ReportDate.objects.create(year=str(y1), month=str(m1))

        flag = 0
        contracts = Contract.objects.exclude(contract__exact='test').filter(iscompleted__exact=False)
        for contract in contracts:
            flag = ContractReportDate.objects.filter(contractid__exact=contract.contractid, 
                                                  dateid__year__exact=y2,
                                                  dateid__month__exact=m2).count()
            if flag == 0:
                ContractReportDate.objects.create(contractid=contract, dateid=date)

        if(m2 == m1):
            loop = 0

    elif(y2 - y1 > 0):
        m1 = m1 + 1
        if(m1 > 12):
            m1 = 1
            y1 = y1 + 1

        date = ReportDate.objects.create(year=str(y1), month=str(m1))

        flag = 0
        contracts = Contract.objects.exclude(contract__exact='test').filter(iscompleted__exact=False)
        for contract in contracts:
            flag = ContractReportDate.objects.filter(contractid__exact=contract.contractid, 
                                                  dateid__year__exact=y2,
                                                  dateid__month__exact=m2).count()
            if flag == 0:
                ContractReportDate.objects.create(contractid=contract, dateid=date)

        if(y2 == y1 and m2 == m1):
            loop = 0

reportDates = ReportDate.objects.all().order_by('-dateid')
serializer = ReportDateSerializerEx(reportDates, many=True)
print(serializer.data)

SyntaxError: 'return' outside function (3106162926.py, line 57)

In [5]:
from contracts.models import Contract
from projects.models import ReportConfirm, ReportDate

contractId = 123476
dateId = 2122
confirmed = 1
userId = 1124

try:
    objects = ReportConfirm.objects.filter(contractid__exact=contractId, dateid__exact=dateId)
    print('objects: ', objects)
    if(objects is not None and len(objects) == 16):
        for obj in objects:
            print('id: ', obj.reportconfirmid)
            obj.pm_c = confirmed
            obj.pmconfirmdate = datetime.now()
            obj.save()

        print('updated')
#     else: 
#         contract = Contract.objects.get(pk=contractId)
#         date = ReportDate.objects.get(pk=dateId)
# #         user = get_user_model().objects.get(pk=userId)
#         ReportConfirm.objects.create(contractid=contract, 
#                                      dateid=date, 
#                                      pm_c=confirmed, 
#                                      pmconfirmdate = datetime.now())
#         print('Inserted')
except Execption as ex:
    print(ex)







objects:  <QuerySet [<ReportConfirm: ReportConfirm object (15004)>]>


In [6]:
from projects.models import ProgressState

ProgressState.objects.filter(contractid__exact=123476, dateid__exact__lte=3121).order_by('-dateid')[:6:-1]


FieldError: Related Field got invalid lookup: exact

In [19]:
from accounts.models import UserRole, RolePermission
from django.contrib.auth import get_user_model

roles = RolePermission.objects.filter(permissionid__permission__exact='Financial Info R/W').values('roleid')
users = UserRole.objects.filter(projectid__exact=123467, roleid__in=roles)

if len(users) == 1:
    print('%s %s' % (users[0].userid.first_name, users[0].userid.last_name))
else:
    print('None')

کاریر تست


In [7]:
from projects.models import Budgetcost
contractId = 123476
dateId = 3123
description = "tetesvbsfdgsdfgbbsfdbsgfbfgb gfbgsgbgbg adbadgfbsdggsbdgdg adfgbsdg"

budget = Budgetcost.objects.get(contractid__exact=contractId, dateid__exact=dateId)
budget.description = description
budget.save()

In [4]:
from projects.serializers import BudgetCostReportSerializer
from projects.models import Budgetcost, ReportDate
contractId = 123476
dateId = 3123

budgetcosts = Budgetcost.objects.filter(contractid__exact=contractId, dateid__lte=dateId).order_by('-dateid')[:6:-1]
serializer = BudgetCostReportSerializer(instance=budgetcosts, many=True)
serializer.data
            

[OrderedDict([('bac_r', 0), ('bac_fc', 0), ('eac_r', 0), ('eac_fc', 0), ('description', ''), ('persianMonth', 'آذر')]), OrderedDict([('bac_r', 0), ('bac_fc', 0), ('eac_r', 0), ('eac_fc', 0), ('description', ''), ('persianMonth', 'دی')]), OrderedDict([('bac_r', 0), ('bac_fc', 0), ('eac_r', 0), ('eac_fc', 0), ('description', ''), ('persianMonth', 'بهمن')]), OrderedDict([('bac_r', 0), ('bac_fc', 0), ('eac_r', 0), ('eac_fc', 0), ('description', ''), ('persianMonth', 'اسفند')]), OrderedDict([('bac_r', 0), ('bac_fc', 200000), ('eac_r', 200000), ('eac_fc', 0), ('description', ''), ('persianMonth', 'فروردین')]), OrderedDict([('bac_r', 0), ('bac_fc', 0), ('eac_r', 0), ('eac_fc', 0), ('description', 'این یک متن تستی برای امتحان کردن می باشد'), ('persianMonth', 'اردیبهشت')])]

In [7]:
from django.db.models import Q
from projects.models import ReportConfirm

try:
#     obj = 
    ReportConfirm.objects.get(Q(contractid__exact=123496, 
                                    dateid__exact=3185, type__exact=9, user_c__exact=1) |
                                    Q(contractid__exact=123496, 
                                    dateid__exact=3185, type__exact=9, sa_c__exact=1))
#     if obj is not None:
        print('1') 
#     else:
#         print('2') 
except ReportConfirm.DoesNotExist:
    print('3') 


IndentationError: unexpected indent (2164067105.py, line 11)

In [9]:
from accounts.models import UserRole
from accounts.serializers import ProjectConfirmersSerializers

contractId = 123478  
userRoles = UserRole.objects.filter(projectid__exact=contractId)
serializer = ProjectConfirmersSerializers(instance=(userRoles[0] if len(userRoles) > 0 else None) if userRoles is not None else None, many=False)
print(serializer.data)


{'project_manager': 'مهندس درخشی', 'financialInfo_confirmor': 'آقای خلیل پور', 'hse_confirmor': 'آقای خلیل پور', 'progressState_confirmor': 'آقای خلیل پور', 'timeProgressState_confirmor': 'آقای خلیل پور', 'invoice_confirmor': 'آقای خلیل پور', 'financialInvoice_confirmor': '', 'workVolume_confirmor': 'آقای خلیل پور', 'pmsProgress_confirmor': 'آقای خلیل پور', 'budget_confirmor': '', 'machinary_confirmor': 'آقای خلیل پور', 'projectPersonel_confirmor': 'آقای خلیل پور', 'problem_confirmor': 'آقای خلیل پور', 'criticalAction_confirmor': 'آقای خلیل پور', 'projectDox_confirmor': 'آقای خلیل پور', 'periodicDox_confirmor': 'آقای خلیل پور', 'zoneImage_confirmor': 'آقای خلیل پور'}


In [7]:
from contracts.models import *
from projects.models import *
from projects.serializers import TimeProgressStateSerializer

userId = 1193
contractId = 123480
dateId = 2129
reportId = 4

#SetReportVisit(userId, contractId, dateId, reportId)            
x = TimeprogressState.objects.filter(contractid__exact=contractId, dateid__exact=dateId)
print("len(x): ", len(x))

flg = 1 if x.count() > 0 else 0
print("1")
if flg == 0:
    contract = Contract.objects.get(pk=contractId)
    date = ReportDate.objects.get(pk=dateId)
    TimeprogressState.objects.update_or_create(contractid=contract ,dateid=date, 
                                               defaults={'plan_replan':'00', 'eep_date':datetime.now(), 'eee_date':datetime.now(), 
                                               'epp_date':datetime.now(), 'epe_date':datetime.now(), 'ecp_date':datetime.now(), 
                                               'ece_date':datetime.now(), 'epjp_date':datetime.now(), 'epje_date':datetime.now()})
print("2")
timeProgressStates = TimeprogressState.objects.filter(contractid__exact=contractId, dateid__lt=dateId).order_by('dateid')
# for item in timeProgressStates:
#     print(item)
print(timeProgressStates)
print("3")
serializer = TimeProgressStateSerializer(instance=timeProgressStates, many=True)
print("4")
print(serializer.data)
print("5")

len(x):  2
1
2
<QuerySet [<TimeprogressState: TimeprogressState object (1628)>, <TimeprogressState: TimeprogressState object (1627)>, <TimeprogressState: TimeprogressState object (1641)>]>
3
4


DoesNotExist: ReportDate matching query does not exist.

In [15]:
from projects.models import Machinary
from django.db.models import F, Q, Sum, IntegerField, ExpressionWrapper
from django.db.models.expressions import Window

# Machinary.objects.filter(contractid__exact=123479, dateid=1126).annotate(
#     activno=Sum('activeno')).values('activno')

activNo = Machinary.objects.filter(contractid__exact=123479, dateid=1126).aggregate(
    Sum('activeno'))
# m = 'total no'

print('activNo: ', activNo)
print("----------------------------")
X=Machinary.objects.filter(contractid__exact=123479, dateid=1126).values('machinaryid').annotate(
    activno=Sum(ExpressionWrapper(F('activeno'), output_field=IntegerField()))).values( 'activno')
for item in X:
    print(item)
print("----------------------------")
Machinary.objects.filter(contractid__exact=123479, dateid=1126).annotate(
    activno=Sum('activeno')
#     machin=F(m)
).values( 'activno')


activNo:  {'activeno__sum': 85}
----------------------------
{'activno': 2}
{'activno': 2}
{'activno': 1}
{'activno': 4}
{'activno': 24}
{'activno': 1}
{'activno': 2}
{'activno': 3}
{'activno': 11}
{'activno': 1}
{'activno': 14}
{'activno': 3}
{'activno': 1}
{'activno': 3}
{'activno': 3}
{'activno': 6}
{'activno': 4}
----------------------------


<QuerySet [{'activno': 2}, {'activno': 2}, {'activno': 1}, {'activno': 4}, {'activno': 24}, {'activno': 1}, {'activno': 2}, {'activno': 3}, {'activno': 11}, {'activno': 1}, {'activno': 14}, {'activno': 3}, {'activno': 1}, {'activno': 3}, {'activno': 3}, {'activno': 6}, {'activno': 4}]>